# Исследование надежности заемщиков


**Цель:** на основе статистики о платёжеспособности клиентов исследовать, какие факторы влияют на факт возврата кредита в срок.

**План работы:**
 - изучить данные 
 - провести исследовательский анализ данных, понять, как различные факторы влияют на возврат кредита в срок

Посмотрим на данные.

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

Проверим наличие пропусков.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Один из столбцов с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можем медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` заполним медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Отрицательное количество дней трудового стажа в столбце `days_employed` это аномалия. Заменим все отрицательные значения положительными.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Посмотрим на уникальные значения столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим эти строки.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Убедимся, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype(int)

Обработаем неявные дубликаты в столбце `education`.

In [14]:
data['education'] = data['education'].str.lower()

Проверим количество дубликатов и удалим, если такие имеются.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

## Категоризация данных

Категоризируем данные, создав в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`
- 30001–50000 — `'D'`
- 50001–200000 — `'C'`
- 200001–1000000 — `'B'`
- 1000001 и выше — `'A'`

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрим на цели взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`
- `'операции с недвижимостью'`
- `'проведение свадьбы'`
- `'получение образования'`

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# создадим сводную таблицу по данным, которые у нас есть
data_pivot_children = data.pivot_table(index='children', columns='debt', values='education', aggfunc='count')

# добавим новую колонку с конверсией (% людей с задолженностью от общего числа заемщиков категории)
data_pivot_children['conversion'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1])

# заполним пропуски и выведем результат в % отсортированный по возрастанию
data_pivot_children = data_pivot_children.fillna(0).sort_values(by='conversion')
data_pivot_children = data_pivot_children.style.format({'conversion': '{:,.0%}'.format})
data_pivot_children

debt,0,1,conversion
children,,,
5,9.000000,0.000000,0%
0,13028.000000,1063.000000,8%
3,303.000000,27.000000,8%
1,4364.000000,444.000000,9%
2,1858.000000,194.000000,9%
4,37.000000,4.000000,10%


In [23]:
data.pivot_table(index='children', values='debt', aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


In [24]:
# убедимся, что посчитали правильно и в изначальном датафрейме действительно не было людей с 5 детьми и задолженностью
data.loc[(data['children'] == 5) & (data['debt'] == 1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


**Вывод:** 

Глядя на данные о конверсии можно сказать, что с увеличением количества детей, растет процент людей с наличием задолженностей. 

Также видим, что те, у кого 5 детей 100% вовремя выплачивают кредиты, а в семьях с 3 детьми процент схож с семьями, где нет детей. Тут стоит обратить внимание на то, что количество людей в выборке очень маленькое, может быть не показательным.

При этом есть теория о том, что такие люди могут получить дополнительные пособия или выплаты (за третьего ребенка и за пятого), что помогает им избежать задолженностей.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
data_pivot_family = data.pivot_table(index='family_status', columns='debt', values='education', aggfunc='count')
data_pivot_family['conversion'] = data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1])
data_pivot_family = data_pivot_family.sort_values(by='conversion')
data_pivot_family = data_pivot_family.style.format({'conversion': '{:,.0%}'.format})
data_pivot_family

debt,0,1,conversion
family_status,,,
вдовец / вдова,888,63,7%
в разводе,1105,84,7%
женат / замужем,11334,927,8%
гражданский брак,3749,385,9%
Не женат / не замужем,2523,273,10%


**Вывод:** 

Меньше задолженностей, судя по результатам, имеют люди в разводе или вдовцы. В данном случае эти категории схожи, поэтому возможно имеет смысл объеденить их, тогда количество людей будет более схоже в разных категориях. 

При этом хуже всего себя показывают люди не состоящие в браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
data_pivot_income = data.pivot_table(index='total_income_category', columns='debt', values='education', aggfunc='count')
data_pivot_income['conversion'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1])
data_pivot_income = data_pivot_income.sort_values(by='conversion')
data_pivot_income = data_pivot_income.style.format({'conversion': '{:,.0%}'.format})
data_pivot_income

debt,0,1,conversion
total_income_category,,,
D,328,21,6%
B,4660,354,7%
A,23,2,8%
C,14568,1353,8%
E,20,2,9%


**Вывод:** 

Как ни странно меньший процент людей с задолженностью в категории D, что соответствует невысокому уровню дохода. 

Хотя, опять же, в категории с самым высоким и самым низким уровнем дохода не так много людей, чтобы считать результаты показательными. Возможно стоит немного изменить условия категоризации (разделить на меньшее количество категорий). 

### Как разные цели кредита влияют на его возврат в срок?

In [27]:
data_pivot_purpose = data.pivot_table(index='purpose_category', columns='debt', values='education', aggfunc='count')
data_pivot_purpose['conversion'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1])
data_pivot_purpose = data_pivot_purpose.sort_values(by='conversion')
data_pivot_purpose = data_pivot_purpose.style.format({'conversion': '{:,.0%}'.format})
data_pivot_purpose

debt,0,1,conversion
purpose_category,,,
операции с недвижимостью,9971,780,7%
проведение свадьбы,2130,183,8%
получение образования,3619,369,9%
операции с автомобилем,3879,400,9%


**Вывод:** 

Лучше всего в плане своевременной оплаты без задолженностей себя показывает группа людей, желающих получить кредит на операции с недвижимостью. 

Можно объяснить это тем, что покупка недвижимости - более серьезный и дорогостоящий шаг, чем свадьба и автомобиль.Скорее всего люди оказываются более подготовлены к этому, тщательнее всё продумывают.

## Возможные причины появления пропусков в исходных данных.

In [28]:
data.income_type.unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Пропуски наблюдались только в столбцах с информацией про доходы и стаж. Если мы посмотрим на значения, представленные в столбце "тип дохода", то увидим, что среди заемщиков есть студенты, безработные, пенсионеры, в декрете. Скорее всего некоторым из низ просто нечего указать в этих колонках. Можно будет сказать точнее, если посмотреть на группы людей, у которых они наблюдаются, если есть такой запрос.

Но еще возникает вопрос логический, как они получили займы, не имея дохода. Этот ответ можно получить, если узнать логику предоставления кредитов в организации (на основании каких сведений). Если поля `доход` и `стаж` обязательны к заполнению при подаче заявки, то может иметь место какой-то баг, в результате которого информация теряется.

# Общий вывод

**Основные этапы исследования надежности заемщиков:**
    
1. Были выгружены и изучены данные о более чем 20 тыс. клиентов банка, содержащий следующую информацию о клиентах: `возраст, пол, образование, семейное положение, количество детей, тип занятости, трудовой стаж, ежемесячный доход, цель кредита, наличие задолженности`.

2. В ходе изучения был выявлен ряд проблем:
    - отсутсвие информации о `стаже работы` и `ежемесячном доходе` в 2174 строках
    - отрицательные и аномальные (20) значения в данных о `количестве детей`
    - отрицательные значения в данных о `трудовом стаже`
    - неявные дубликаты в данных об `образовании`
    
3. Была проведена предобработка данных для устранения выявленных проблем и предотвращения их влияния на точность исследования, а именно:
    - пропуски значений в данных о `ежемесячном доходе` были заполнены медианными значениями в соответствии с типом занятости, так как тип занятости влияет на ежемесячный доход. Медианные значения помогают максимально сгладить погрешности вычисления, связанные с большими выбросами (очень высокие или очень низкие значения)
    - пропуски значений в данных о `трудовом стаже` также были заполнены медианными значениями в соответствии с типом занятости
    - аномальные значения в данных о `количестве детей` были удалены, а отрицательные значения в данных о `трудовом стаже` были заменены на положительные
    - неявные дубликаты в данных об `образовании` были удалены путем приведения всех значений к нижнему регистру
    
4. Были систематизированы данные о `ежемесячном доходе` клиентов путем присвоения клиентам определенной категории (добавлен новый столбец `категории ежемесячного дохода`:
    - до 30000 руб/мес — 'E';
    - 30001–50000 руб/мес — 'D';
    - 50001–200000 руб/мес — 'C';
    - 200001–1000000 руб/мес — 'B';
    - 1000001 и выше руб/мес — 'A'.
    
5. Аналогично были систематизированы данные о `цели кредита` клиентов путем присвоения клиентам определенной категории (добавлен новый столбец `категории целей кредита` с четыремя группами:
    - 'операции с автомобилем'
    - 'операции с недвижимостью'
    - 'проведение свадьбы'
    - 'получение образования'

**Основной вывод по исследованию надежности заемщиков.**

Количество детей, семейное положение, уровень дохода и цели кредита влияют на возврат кредита в срок.
    
**Детали.**

Высокую платежеспособность продемонстрировали:

- люди, у которых нет детей, а также с тремя и пятью детьми
- люди в разводе, а также вдовы/вдовцы
- люди с уровнем дохода категории D (30001–50000 руб/мес)
- люди, желающие получить кредит на операции с недвижимостью

Данные рекомендации стоит учитывать при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.